<a href="https://colab.research.google.com/github/Soedirman-Machine-Learning/Emotion-detection/blob/main/VGG16%2BRes10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Menyimpan hasil ekstrak ke dalam folder datasets
import zipfile,os
local_zip = "/content/drive/MyDrive/Colab Notebooks/Dataset_Emosi.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("/content/datasets")
zip_ref.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import os
from skimage.io import imread
from skimage.transform import resize
import cv2
from keras.models import Sequential

 
# Mengihitung waktu lamanya eksekusi tiap sel di Google Colab
!pip install ipython-autotime
 
%load_ext autotime

time: 137 µs (started: 2021-01-24 16:56:22 +00:00)


In [ ]:
print(tf.__version__)

2.4.0
time: 2.72 ms (started: 2021-01-24 16:56:22 +00:00)


In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255, 
    validation_split = 0.2,
    rotation_range = 20,
    horizontal_flip = True,
    shear_range = 0.2,
    zoom_range = 0.2,    
    vertical_flip = True,
    fill_mode = "nearest")

time: 1.57 ms (started: 2021-01-24 16:56:22 +00:00)


In [ ]:
#Memuat dataset pelatihan & validasi
base_dir = os.path.join("/content/datasets/Dataset_Emosi/Train")
 
img_size =224
batch_size = 128
 
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size, 
    subset='training')
 
val_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size, 
    subset='validation')

 
#Memuat dataset pengujian
X_test = []
y_test = []
labels = ['Marah',
          'Sedih',
          'Senang']
 
for i,label in enumerate(labels):
    folder = os.path.join("/content/datasets/Dataset_Emosi/Test",label)
    files = sorted(os.listdir(folder))
    files = [x for x in files if x.endswith(".jpg")]
    for k,file in enumerate(files):
        image_path = os.path.join(folder, file)       
        image = imread(image_path)/255.
        image = resize(image,(224,224))
        X_test.append(image)
        category = os.path.split(folder)[-1]
        y_test.append(i)
 
X_test = np.array(X_test)
y_test = np.array(y_test)


 
#Menampilkan bentuk dari masing-masing dataset
for image_batch, label_batch in train_generator:
  break
print("Bentuk array dari dataset train (pelatihan) adalah:", image_batch.shape,label_batch.shape)
for image_batch, label_batch in val_generator:
  break
print("Bentuk array dari dataset validation (validasi) adalah:", image_batch.shape,label_batch.shape)
print("Bentuk array dari dataset test (pengujian) adalah:", X_test.shape,y_test.shape,)

Found 654 images belonging to 3 classes.
Found 162 images belonging to 3 classes.
Bentuk array dari dataset train (pelatihan) adalah: (128, 224, 224, 3) (128, 3)
Bentuk array dari dataset validation (validasi) adalah: (128, 224, 224, 3) (128, 3)
Bentuk array dari dataset test (pengujian) adalah: (204, 224, 224, 3) (204,)
time: 7 s (started: 2021-01-24 16:56:22 +00:00)


In [ ]:
print (train_generator.class_indices)

labels_txt = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt', 'w') as f:
  f.write(labels_txt)

{'Marah': 0, 'Sedih': 1, 'Senang': 2}
time: 3.9 ms (started: 2021-01-24 16:56:29 +00:00)


In [ ]:
IMG_SHAPE = (224,224, 3)
# Membuat model dasar (base model) dari pre-trained model 
base_model = tf.keras.applications.VGG16(input_shape=IMG_SHAPE,
                                          include_top=False,
                                          weights='imagenet')

58892288/58889256 [==============================] - 0s 0us/step
time: 6.16 s (started: 2021-01-24 16:56:29 +00:00)


In [ ]:
base_model.trainable = False
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
model = tf.keras.Sequential([
                             base_model,
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(512, activation='relu'),
                             tf.keras.layers.Dropout(rate=0.25),
                             tf.keras.layers.Dense(3, activation='softmax')
])

time: 69.6 ms (started: 2021-01-24 16:56:35 +00:00)


In [ ]:
model.compile("adam",loss="categorical_crossentropy",metrics=["acc"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               12845568  
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
Total params: 27,561,795
Trainable params: 12,847,107
Non-trainable params: 14,714,688
_________________________________________________________________
time: 23.3 ms (started: 2021-01-24 16:56:35 +00:00)


In [ ]:
#es = EarlyStopping(monitor="val_loss", patience=7, verbose=1, min_delta=0.09, mode="auto")
history = model.fit(train_generator,
                    epochs=50,
                    validation_data=val_generator)

Epoch 1/50
6/6 [==============================] - 22s 2s/step - loss: 9.2136 - acc: 0.3510 - val_loss: 1.7075 - val_acc: 0.5309
Epoch 2/50
6/6 [==============================] - 10s 2s/step - loss: 1.8586 - acc: 0.4468 - val_loss: 1.5583 - val_acc: 0.4259
Epoch 3/50
6/6 [==============================] - 10s 2s/step - loss: 1.4386 - acc: 0.5099 - val_loss: 1.0773 - val_acc: 0.5494
Epoch 4/50
6/6 [==============================] - 10s 2s/step - loss: 1.0228 - acc: 0.5990 - val_loss: 0.6961 - val_acc: 0.6975
Epoch 5/50
6/6 [==============================] - 10s 2s/step - loss: 0.8394 - acc: 0.6511 - val_loss: 1.0053 - val_acc: 0.5988
Epoch 6/50
6/6 [==============================] - 10s 2s/step - loss: 0.9442 - acc: 0.6069 - val_loss: 0.8468 - val_acc: 0.5679
Epoch 7/50
6/6 [==============================] - 10s 2s/step - loss: 0.7868 - acc: 0.6257 - val_loss: 0.8808 - val_acc: 0.5926
Epoch 8/50
6/6 [==============================] - 10s 2s/step - loss: 0.7038 - acc: 0.7052 - val_loss: 0

In [ ]:
fig = plt.figure(figsize=(7, 4))
fig.set_figheight(10)
fig.set_figwidth(15)
 
plt.subplot(2, 2, 1)
plt.plot(history.history["acc"], label = "Training Accuracy")
plt.plot(history.history["val_acc"], label="Validation Accuracy")
plt.legend()
plt.title("Kurva Tingkat Akurasi", size=15)
plt.grid(zorder=0)
 
plt.subplot(2, 2, 2)
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()
plt.title("Kurva Tingkat Error", size=15)
plt.grid(zorder=0)
plt.show()

In [ ]:
print(train_generator)

In [ ]:
y_test2 = to_categorical(y_test)
X_test3, y_test3 = (X_test, y_test2)

In [ ]:
#Menampilkan matriks yang benar dan matriks hasil prediksi

#Label yang benar
y_true = np.argmax(y_test2,axis=1)

#Label prediksi
Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print(y_true)
print(y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if not title:
    if normalize:
      title = 'Normalized confusion matrix'
    else:
      title = 'Confusion matrix, without normalization'

  #compute confusion matrix
  cm = confusion_matrix(y_true, y_pred)
  #Only use the labels that appear in the data
  #classes = classes[unique_labels(y_true, y_pred)]
  if normalize:
    cm = cm.astype('float') / cm.sum(axis=1)[:, np. newaxis]
    print("Normalized confusion matrix")
  else:
    print('Confusion matrix, without normalization')

  print(cm)

  fig, ax = plt.subplots(figsize=(5,5))
  im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
  #ax.figure.colorbar(im, ax=ax)
  #we want to show all ticks...
  ax.set(xticks=np.arange(cm.shape[1]),
         yticks=np.arange(cm.shape[0]),
         #...and label them with the respective list entries
         xticklabels=classes, yticklabels=classes,
         title=title,
         ylabel='Label Benar',
         xlabel='Label Prediksi')
  
  #Rotate the tick labels and set their alignment.
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
           rotation_mode="anchor")
  #Loop over data dimensions and create text annotations.
  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
      ax.text(j, i, format(cm[i, j], fmt),
              ha="center", va="center",
              color="white" if cm[i, j] > thresh else "black")
      fig.tight_layout()
      return ax

np.set_printoptions(precision=2)

plot_confusion_matrix(y_true, y_pred, classes=labels, normalize=True,
                      title='Normalized confusion matrix')

In [ ]:
saved_model_dir = 'save/model'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open('Emotion_Detection_MobileNet.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
saved_model_dir = 'save/model'
tf.saved_model.save(model, saved_model_dir)

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

with open('Emotion_Detection_MobileNet_Quantized', 'wb') as f:
  f.write(tflite_model)

In [ ]:
n = 196

plt.imshow(X_test[n])
plt.show()

true_label = np.argmax(y_test2,axis=1) [n]
print("Label yang benar adalah:", true_label,":", labels[true_label])
prediction = model.predict(X_test[n][np.newaxis,...])[0]
print("Nilai yang diprediksi adalah:",prediction)
predicted_label = np.argmax(prediction)
print("Label yang diprediksi adalah:",predicted_label,":",labels[predicted_label])

if true_label == predicted_label:
  print("Prediksi benar")
else:
  print("Prediksi salah")

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/PENGUJIAN/senyum14.jpeg')
orig = image.copy()
(h,w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 1.0, (300,300),
                             (104.0, 177.0, 123.0))

net = cv2.dnn.readNet('/content/drive/MyDrive/Colab Notebooks/Face_detector/deploy.prototxt', '/content/drive/MyDrive/Colab Notebooks/Face_detector/res10_300x300_ssd_iter_140000.caffemodel')

#melewatkan blob melalui jaringan dan mendapatkan deteksi wajah
print("Mendeteksi Wajah...")
net.setInput(blob)
detections = net.forward()

for i in range(0, detections.shape[0]):
  #ekstrak keyakinan (probabilitas) yang terkait dengan deteksi
  confidence = detections[0, 0, i, 2]
  
  if confidence > 0.5:
    box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
    (startX, startY, endX, endY) = box.astype("int")

    (startX, startY) = (max(0, startX), max(0, startY))
    (endX, endY) = (min(w-1, endX), min(h-1, endY))
  
  # Ekstrak ROI wajah, konversikan dari BGR ke pemesanan saluran RGB,
  # dan mengubah ukurannya menjadi 224x224, dan lalu pre-proses
  face = image [startY:endY, startX:endX]
  face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
  face = cv2.resize(face, (224, 224))
  face = img_to_array(face)
  face = preprocess_input(face)
  face = np.expand_dims(face, axis=0)
  
  # Membaca wajah dengan model
  (Marah, Sedih, Senang) = model.predict(face)[0]
  
  # Marah warna merah, sedih warna hijau dan senang warna biru
  if (Marah>Sedih and Marah>Senang):
    label = "marah"
  elif (Sedih>Senang and Sedih>Marah):
    label = "sedih"
  elif (Senang>Sedih and Senang>Marah):
    label = "senang"

  if label == "sedih":
    color = (0, 255, 0)
  elif label == "marah":
    color = (0, 0, 255)
  elif label == "senang":
    color = (255, 0, 0)
  
  # Probabilitas hasil deteksi
  label = "{}: {:.2f}%".format(label, max(Marah, Sedih, Senang) * 100)
  
  # Menampilkan hasil dengan label dan kotak
  cv2.putText(image, label, (startX, startY - 10),
              cv2.FONT_HERSHEY_TRIPLEX, 1, color, 2)
  cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
  
# Menampilkan output
cv2_imshow(image)
cv2.waitKey(0)

In [ ]:
# Mengimport lib
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os


def detect_and_predict_emotion(frame, faceNet, emotionNet):
  #membuat dimensi kotak deteksi
  (h, w) = frame.shape[:2]
  blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
                               (104.0, 177.0, 123.0))
 
  # Melewatkan blob dan mendeteksi wajah
  faceNet.setInput(blob)
  detections = faceNet.forward()
 
  # Inisialisasi
  faces = []
  locs = []
  preds = []
 
  for i in range(0, detections.shape[2]):
    # ekstrak keyakinan (yaitu, probabilitas) yang terkait dengan deteksi
    confidence = detections[0, 0, i, 2]
    
    if confidence > 0.5:
      # Menghitung koordinat (x, y) dari kotak pembatas untuk objek
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (startX, startY, endX, endY) = box.astype("int")
 
      # Memastikan kotak pembatas berada dalam dimensi bingkai
      (startX, startY) = (max(0, startX), max(0, startY))
      (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
 
      # Ekstrak ROI wajah, konversikan dari BGR ke pemesanan saluran RGB,
      # dan mengubah ukurannya menjadi 224x224, dan lalu pre-proses
      face = frame[startY:endY, startX:endX]
      face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
      face = cv2.resize(face, (224, 224))
      face = img_to_array(face)
      face = preprocess_input(face)
 
      # Menambahkan kotak deteksi
      faces.append(face)
      locs.append((startX, startY, endX, endY))
      
    if len(faces) > 0:
      faces = np.array(faces, dtype="float32")
      preds = emotionNet.predict(faces, batch_size=128)
    return (locs, preds)
 
faceNet=cv2.dnn.readNet('/content/drive/MyDrive/Colab Notebooks/Face_detector/deploy.prototxt','/content/drive/MyDrive/Colab Notebooks/Face_detector/res10_300x300_ssd_iter_140000.caffemodel')

In [ ]:
import base64
import html
import io
import time
 
from IPython.display import display, Javascript
from google.colab.output import eval_js
import numpy as np
from PIL import Image
import cv2
 
def start_input():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 512, 512);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'Ketika selesai, klik disini atau pada video untuk berhenti dari demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 512; //video.videoWidth;
      captureCanvas.height = 512; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function takePhoto(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
 
  display(js)
  
def take_photo(label, img_data):
  data = eval_js('takePhoto("{}", "{}")'.format(label, img_data))
  return data

In [ ]:
def js_reply_to_image(js_reply):
    """
    input: 
          js_reply: JavaScript object, contain image from webcam
    output: 
          image_array: image array RGB size 512 x 512 from webcam
    """
    jpeg_bytes = base64.b64decode(js_reply['img'].split(',')[1])
    image_PIL = Image.open(io.BytesIO(jpeg_bytes))
    image_array = np.array(image_PIL)
 
    return image_array

In [ ]:
start_input()
label_html = 'Capturing...'
img_data = ''
count = 0 
from google.colab.patches import cv2_imshow
while True:
  js_reply = take_photo(label_html, img_data)
  if not js_reply:
    break
  
  image = js_reply_to_image(js_reply)
  
  # Mengambil frame dari aliran video berulir dan 
  # ukurannya maksimum lebar 400 pixel
  frame = image
  v=True
  if v == True:
    
    frame = imutils.resize(frame, width=400)
    
    # Mendeteksi emosi
    (locs, preds) = detect_and_predict_emotion(frame, faceNet, model)
    for (box, pred) in zip(locs, preds):
      
      # Membuka kotak dan prediksi
      (startX, startY, endX, endY) = box
      (Marah, Sedih, Senang) = pred

      # Menggunakan masker hijau, tidak bermasker merah
      if (Marah>Sedih and Marah>Senang):
        label = "sedih"
      elif (Sedih>Senang and Sedih>Marah):
        label = "marah"
      elif (Senang>Marah and Senang>Sedih):
        label = "senang"
        
      if label == "marah":
        color = (0, 255, 0)
      elif label == "sedih":
        color = (0, 0, 255)
      elif label == "senang" :
        color = (255, 0, 0)
        
      # Probabilitas pada label
      label = "{}: {:.2f}%".format(label, max(Marah, Sedih, Senang) * 100)
      
      # Menampilkan hasil dengan label dan kotak dari frame
      frame=cv2.putText(frame, label, (startX, startY - 10),cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
      frame=cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
      # Menampilkan ouput
      cv2_imshow(frame)